In [16]:
import pandas as pd
import numpy as np
import sys, os, re
import matplotlib.pyplot as plt

# from importlib import reload

if re.match(r'^(\w\:\\)|(/)', os.getcwd()):
    # sys.path.insert(0, "c:/Users/dmitr/devs/quantl/")
    # sys.path.insert(0, "c:/Users/dmitr/devs/Qube/")
    sys.path.insert(0, "c:/Users/dmitr/devs/Qube2exp/src/")
    sys.path.insert(0, "c:/Users/dmitr/devs/Qube2exp/tests/")
    print('installed')


# just some handy helpers from old Qube
from ta_helpers import kama, kama_indicator, apply_to_frame

%load_ext cython

T = pd.date_range('2024-01-01 00:00', freq='1Min', periods=100_000)
ds = 1 + (2*np.random.randn(len(T))).cumsum()
data = list(zip(T, ds))

installed
The cython extension is already loaded. To reload it, use:
  %reload_ext cython


## Cython v2 tests (cseries1)

In [2]:
%load_ext autoreload
%autoreload 2

from qube.utils import reload_pyx_module

reload_pyx_module('../../src/qube/core/')
from qube.core.series import TimeSeries
del sys.modules['qube.core.series']
from qube.core.series import (TimeSeries, Sma, Ema, recognize_time, Tema, Dema, Kama, OHLCV, RollingSum)

 > Reloaded ../../src/qube/core\series.pyx
 > Reloaded c:\Users/dmitr/devs/Qube2exp/src\qube\core\series.pyx


## Series example

In [11]:
# %%timeit
ts = TimeSeries('test', '10Min')
ts.update(recognize_time('2024-01-01 00:00'), 1)
ts.update(recognize_time('2024-01-01 00:01'), 5)
ts.update(recognize_time('2024-01-01 00:06'), 2)
ts.update(recognize_time('2024-01-01 00:12'), 3)
ts.update(recognize_time('2024-01-01 00:21'), 4)
ts.update(recognize_time('2024-01-01 00:22'), 5)
ts.update(recognize_time('2024-01-01 00:31'), 6)
ts.update(recognize_time('2024-01-01 00:33'), 7)
ts.update(recognize_time('2024-01-01 00:45'), -12)
ts.update(recognize_time('2024-01-01 00:55'), 12)
ts.update(recognize_time('2024-01-01 01:00'), 12)

1

In [13]:
ts.to_series()

2024-01-01 00:00:00     2.0
2024-01-01 00:10:00     3.0
2024-01-01 00:20:00     5.0
2024-01-01 00:30:00     7.0
2024-01-01 00:40:00   -12.0
2024-01-01 00:50:00    12.0
2024-01-01 01:00:00    12.0
Name: test, dtype: float64

## <font color='orange'>WIP</font>

In [24]:
# %%timeit 
ts = TimeSeries('close', '1h')
s1 = Sma(ts, 50)
e1 = Ema(ts, 50)
ss1 = Sma(s1, 50)
ee1 = Ema(e1, 50)
t1 = Tema(ts, 50)
k1 = Kama(ts, 50)
[ts.update(ti.asm8, vi) for ti, vi in data];

1.04 s ± 30.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
k1s = k1.to_series().dropna()
ik1 = apply_to_frame(kama_indicator, ts.to_series(), 50).dropna()
sum(ik1 - k1.to_series().dropna()) < 1e-8

True

In [19]:
k1 = Kama(ts, 50)
k1s = k1.to_series().dropna()
sum(ik1 - k1.to_series().dropna()) < 1e-8

True

In [ ]:
plt.plot(ts.to_series(), lw=0.5)
plt.plot(Tema(ts, 50).to_series())
plt.plot(Dema(ts, 50).to_series())
plt.plot(t1.to_series())

In [23]:
# %%timeit
x0 = ts.to_series()
pe1 = x0.ewm(span=50, min_periods=50).mean()
pee1 = pe1.ewm(span=50, min_periods=50).mean()
ps1 = x0.rolling(50).mean()
pss1 = ps1.rolling(50).mean()

2.32 ms ± 58 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
plt.plot(x0, lw=0.5)
plt.plot(s1.to_series()); plt.plot(ps1)
plt.plot(ss1.to_series()); plt.plot(pss1)
plt.plot(e1.to_series()); plt.plot(pe1)
plt.plot(ee1.to_series()); plt.plot(pee1)
# plt.plot(s1.to_series()-ps1)

In [31]:
# %%timeit
s1 = Sma(ts, 50)
e1 = Ema(ts, 50)
ss1 = Sma(s1, 50)
ee1 = Ema(e1, 50)
k1 = Kama(e1, 50)

In [ ]:
plt.plot(e1.to_series()); plt.plot(k1.to_series())

### indicators work ... 

In [42]:
ts0 = TimeSeries('close', '1Min')
# em0 = Sma(ts0, 3)
# em1 = Sma(em0, 3)
k0 = Kama(ts0, 6)  

for t, v in [
    ('2024-01-01 00:00', 9), ('2024-01-01 00:00', 1),
    ('2024-01-01 00:01', 2), ('2024-01-01 00:01', 3),   ('2024-01-01 00:01', 2),
    ('2024-01-01 00:02', 3),
    ('2024-01-01 00:03', 4),
    ('2024-01-01 00:04', 5), ('2024-01-01 00:04', 5.1), ('2024-01-01 00:04:20', 5),
    ('2024-01-01 00:05', 6), ('2024-01-01 00:05', 7),   ('2024-01-01 00:05', 6),
    ('2024-01-01 00:07', 8), ('2024-01-01 00:07', -1),  ('2024-01-01 00:07', 8),
    ('2024-01-01 00:08', 8),
    ('2024-01-01 00:09', 8),
    ('2024-01-01 00:10', 12),
    ('2024-01-01 00:11', 13),
    ('2024-01-01 00:12', 14),
    ('2024-01-01 00:13', 15),
    ('2024-01-01 00:14', 17),
    ('2024-01-01 00:15', 4),
]:
    ts0.update(recognize_time(t), v)

In [43]:
k0.to_series()

2024-01-01 00:00:00          NaN
2024-01-01 00:01:00          NaN
2024-01-01 00:02:00          NaN
2024-01-01 00:03:00          NaN
2024-01-01 00:04:00          NaN
2024-01-01 00:05:00     6.000000
2024-01-01 00:07:00     6.888889
2024-01-01 00:08:00     7.382716
2024-01-01 00:09:00     7.657064
2024-01-01 00:10:00     9.587258
2024-01-01 00:11:00    11.104032
2024-01-01 00:12:00    12.391129
2024-01-01 00:13:00    13.550627
2024-01-01 00:14:00    15.083682
2024-01-01 00:15:00    14.748120
Name: kama(6,2,30), dtype: float64

### OHLC tests

In [3]:
ohlc = OHLCV('1Min')
s1 = Sma(ohlc.close, 10)
for t, v in [
    ('2024-01-01 00:00', 9), ('2024-01-01 00:00', 1),
    ('2024-01-01 00:01', 2), ('2024-01-01 00:01', 3),   ('2024-01-01 00:01', 2),
    ('2024-01-01 00:02', 3),
    ('2024-01-01 00:03', 4),
    ('2024-01-01 00:04', 5), ('2024-01-01 00:04', 5.1), ('2024-01-01 00:04:20', 5),
    ('2024-01-01 00:05', 6), ('2024-01-01 00:05', 7),   ('2024-01-01 00:05', 6),
    ('2024-01-01 00:07', 8), ('2024-01-01 00:07', -1),  ('2024-01-01 00:07', 8),
    ('2024-01-01 00:08', 8),
    ('2024-01-01 00:09', 8),
    ('2024-01-01 00:10', 12), ('2024-01-01 00:10:01', 21),('2024-01-01 00:10:30', 1),('2024-01-01 00:10:31', 5),
    ('2024-01-01 00:11', 13),
    ('2024-01-01 00:12', 14),
    ('2024-01-01 00:13', 15),
    ('2024-01-01 00:14', 17),
    ('2024-01-01 00:15', 4), ('2024-01-01 00:15:10', 12),
]:
    ohlc.update(recognize_time(t), v, 1)
    # r1.update(recognize_time(t), v)

In [49]:
ds = [
    ('2024-01-01 00:00', 9), ('2024-01-01 00:00', 1),
    ('2024-01-01 00:01', 2), ('2024-01-01 00:01', 3),   ('2024-01-01 00:01', 2),
    ('2024-01-01 00:02', 3),
    ('2024-01-01 00:03', 4),
    ('2024-01-01 00:04', 5), ('2024-01-01 00:04', 5.1), ('2024-01-01 00:04:20', 5),
    ('2024-01-01 00:05', 6), ('2024-01-01 00:05', 7),   ('2024-01-01 00:05', 6),
    ('2024-01-01 00:07', 8), ('2024-01-01 00:07', -1),  ('2024-01-01 00:07', 8),
    ('2024-01-01 00:08', 8),
    ('2024-01-01 00:09', 8),
    ('2024-01-01 00:10', 12), ('2024-01-01 00:10:01', 21),('2024-01-01 00:10:30', 1),('2024-01-01 00:10:31', 5),
    ('2024-01-01 00:11', 13),
    ('2024-01-01 00:12', 14),
    ('2024-01-01 00:13', 15),
    ('2024-01-01 00:14', 17),
    ('2024-01-01 00:15', 4), ('2024-01-01 00:15:10', 12),
]

In [48]:
def push(series: TimeSeries, ds: list):
    for t, d in ds:
        if isinstance(t, str):
            t = recognize_time(t)
        if isinstance(d, (list, tuple)):
            series.update(t, d[0], d[1])
        else:
            series.update(t, d)

In [50]:
ohlc = OHLCV('1Min')
push(ohlc, ds)

In [4]:
r1 = RollingSum(3)

In [5]:
r1.update(1, 1); print(r1)
# r1.update(1, 1); print(r1)
# r1.update(1, 0); print(r1)

rs[3] = [0. 1. 0.] @ 1 -> 1


In [51]:
ohlc.to_series()

,open,high,low,close,volume
timestamp,,,,,
2024-01-01 00:00:00,9.0,9.0,1.0,1.0,0.0
2024-01-01 00:01:00,2.0,3.0,2.0,2.0,0.0
2024-01-01 00:02:00,3.0,3.0,3.0,3.0,0.0
2024-01-01 00:03:00,4.0,4.0,4.0,4.0,0.0
2024-01-01 00:04:00,5.0,5.1,5.0,5.0,0.0
2024-01-01 00:05:00,6.0,7.0,6.0,6.0,0.0
2024-01-01 00:07:00,8.0,8.0,-1.0,8.0,0.0
2024-01-01 00:08:00,8.0,8.0,8.0,8.0,0.0
2024-01-01 00:09:00,8.0,8.0,8.0,8.0,0.0


In [ ]:
print(str(ohlc), str(s1))

In [8]:
s1.to_series()

2024-01-01 00:00:00     NaN
2024-01-01 00:01:00     NaN
2024-01-01 00:02:00     NaN
2024-01-01 00:03:00     NaN
2024-01-01 00:04:00     NaN
2024-01-01 00:05:00     NaN
2024-01-01 00:07:00     NaN
2024-01-01 00:08:00     NaN
2024-01-01 00:09:00     NaN
2024-01-01 00:10:00     NaN
2024-01-01 00:11:00     6.2
2024-01-01 00:12:00     7.4
2024-01-01 00:13:00     8.6
2024-01-01 00:14:00     9.9
2024-01-01 00:15:00    10.6
Name: sma(10), dtype: float64

In [45]:
Sma(ohlc.close, 10).to_series()

2024-01-01 00:00:00     NaN
2024-01-01 00:01:00     NaN
2024-01-01 00:02:00     NaN
2024-01-01 00:03:00     NaN
2024-01-01 00:04:00     NaN
2024-01-01 00:05:00     NaN
2024-01-01 00:07:00     NaN
2024-01-01 00:08:00     NaN
2024-01-01 00:09:00     NaN
2024-01-01 00:10:00     5.0
2024-01-01 00:11:00     6.2
2024-01-01 00:12:00     7.4
2024-01-01 00:13:00     8.6
2024-01-01 00:14:00     9.9
2024-01-01 00:15:00    10.6
Name: sma(10), dtype: float64

In [46]:
Sma(ohlc.close, 5).to_series()

2024-01-01 00:00:00     NaN
2024-01-01 00:01:00     NaN
2024-01-01 00:02:00     NaN
2024-01-01 00:03:00     NaN
2024-01-01 00:04:00     3.0
2024-01-01 00:05:00     4.0
2024-01-01 00:07:00     5.2
2024-01-01 00:08:00     6.2
2024-01-01 00:09:00     7.0
2024-01-01 00:10:00     7.0
2024-01-01 00:11:00     8.4
2024-01-01 00:12:00     9.6
2024-01-01 00:13:00    11.0
2024-01-01 00:14:00    12.8
2024-01-01 00:15:00    14.2
Name: sma(5), dtype: float64